# Topology optimization of 2D cantilever beam using SIMP method #

In [2]:
using Gridap, Gridap.CellData, SparseArrays
using NearestNeighbors, Printf  # make sure this is at the top of your file
using Makie,GLMakie, LaTeXStrings, Makie.GeometryBasics

In [ ]:
include("plot_design.jl")
include("OC.jl")
include("SIMP_Inter.jl")
include("filters.jl")

SIMP_Inter (generic function with 1 method)

## Inputs

In [ ]:
L = 60 # Length
W = 20 # Height
domain = (0, L, 0, W) # sizes along X, Y
partition = (L*3, W*3) # mesh sizes
model = CartesianDiscreteModel(domain, partition)
# writevtk(model, "Cantilever_Beam_2D")
# Tag the left face (x = 0); right face (x = L)
labels = get_face_labeling(model)
add_tag_from_tags!(labels, "left", [1, 3, 7]) # 
add_tag_from_tags!(labels, "right", [2, 4, 8]) # 
# Integrating on the domain Ω
degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω, degree)
# neumann boundaries
Γ = BoundaryTriangulation(model, tags="right")
dΓ = Measure(Γ, degree)
# Vector-valued FE space
order = 1
reffe = ReferenceFE(lagrangian, VectorValue{2,Float64}, order)
Vₕ = TestFESpace(Ω, reffe; conformity=:H1, dirichlet_tags="left")
Uₕ = TrialFESpace(Vₕ)
# Constitutive law for LINEAR ELASTICITY 
const E = 119e3 # Young's modulus
const ν = 0.3   # Poisson's ratio
const λ = (E * ν) / ((1 + ν) * (1 - 2 * ν)) # Lamé parameters
const μ = E / (2 * (1 + ν)) # Lamé parameters
ε₀(u) = 0.5 * (∇(u) + transpose(∇(u))) # Strain; In Gridap this can be automatically defined in Gridap.ε
σ(ε₀) = λ * tr(ε₀) * one(ε₀) + 2μ * ε₀ # Stress

σ (generic function with 1 method)

## Filters

In [ ]:

num_ele = num_cells(Ω) # Number of elements
volfrac = 0.4
p = 3
max_iter = 50
filter_radius = 2
H = filters(filter_radius, Ω);

## Run topology optimization

In [ ]:

for iter = 1:max_iter
    if iter == 1
        ρ_new = 0.5 * ones(num_ele)
    end
    ρ_old = copy(ρ_new)
    ρ_new_fil = H * ρ_new # Filter
    # SIMP
    ρ_SIMP, dρ_dp = SIMP_Inter(ρ_new_fil, p, Ω)
    # ===FE-ANALYSIS
    F(x) = VectorValue(0.0, -1) # I want to apply a unit load along Y axis to each node on the right face
    l(v) = ∫(F ⋅ v) * dΓ # Right-hand size
    a(u, v) = ∫(ρ_SIMP * (σ ∘ ε₀(u)) ⊙ ε₀(v)) * dΩ # Left-hand size; (∘) Composite functions
    # Solution of the FE problem
    op = AffineFEOperator(a, l, Uₕ, Vₕ)
    uh = solve(op)
    C = sum(∫(ρ_SIMP * (σ ∘ ε₀(uh)) ⊙ ε₀(uh)) * dΩ) # Compliance
    dC_fil = get_contribution(∫(-dρ_dp * (σ ∘ ε₀(uh)) ⊙ ε₀(uh)) * dΩ, Ω) # Sensitivity of compliance w.r.t rho
    dC = transpose(H) * (dC_fil[:])

    ρ_new = OC(ρ_new, volfrac, dC, num_ele)
    change_obj = maximum(abs.(ρ_new .- ρ_old))
    figs = plot_design(ρ_new, cell_coords, iter, L, W)

    display(figs)

    if iter % 10 == 0
        println("It. ", iter, ", Obj = ", @sprintf("%.5e", C), ", Vol.: ", @sprintf("%.5e", sum(ρ_new) / num_ele), ", ch.: ", @sprintf("%.5e", change_obj))
    end

    if iter == max_iter
        writevtk(Ω, "Cantilever_Beam_2D_TOP", cellfields = ["rho" => ρ_SIMP])
    end

end

It. 10, Obj = 1.51390e+00, Vol.: 3.94883e-01, ch.: 2.00000e-01
It. 20, Obj = 1.07916e+00, Vol.: 3.95214e-01, ch.: 2.00000e-01
It. 30, Obj = 9.91810e-01, Vol.: 4.08679e-01, ch.: 2.00000e-01
It. 40, Obj = 9.91627e-01, Vol.: 4.05310e-01, ch.: 1.93819e-01
It. 50, Obj = 9.57263e-01, Vol.: 4.05725e-01, ch.: 1.70205e-01
